In [337]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import transforms, models
import pytorch_lightning as pl
from pytorch_lightning.utilities.model_summary import ModelSummary
from pytorch_lightning.loggers import WandbLogger
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import wandb
import torchmetrics
import sys
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray import tune
from transformers import DistilBertTokenizer

# Lightning moduel

In [493]:
class textClassifier(pl.LightningModule):
    def __init__(self, vocab_size=10_000, embedding_dim=128, hidden_dim=64, learning_rate=3e-4):
        super().__init__()
        self.save_hyperparameters()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.relu = nn.Sequential(
        nn.Linear(hidden_dim, 1),
        nn.ReLU()
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.embedding(x)
#         x = nn.utils.rnn.pack_padded_sequence(x, text_lengths, batch_first=True)
        x, (hidden, cell) = self.lstm(x)
        x = hidden.squeeze(0)
        x = self.relu(x)
        y_hat = self.sigmoid(x)
        
        return y_hat
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        print('y_hat:', y_hat.dtype)
        print('y:', y.dtype)
        loss = F.binary_cross_entropy(y_hat, y)
        pred = loss.round()
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return {'loss':loss, 'pred':pred}
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        print('y_hat:', y_hat.dtype)
        print('y:', y.dtype)
        loss = F.binary_cross_entropy(y_hat, y)
        pred = loss.round()
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return {'val_loss':loss, 'pred':pred}
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adamax(self.parameters(), lr=self.hparams.learning_rate)
        return [optimizer]    

# Dataset

In [494]:
import contractions
from bs4 import BeautifulSoup
from unidecode import unidecode
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

class textDataset(Dataset):
    def __init__(self, data_dir):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.tweets = pd.read_csv(data_dir)
        self.tweets['keyword'] = self.tweets['keyword'].fillna('')
        self.tweets['tweet'] = self.tweets['keyword'] + ' ' + self.tweets['text']
        self.tweets = self.text_preprocessing(self.tweets)
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    
    def rem_urls(self, data):
        regex = re.compile(f"https?://(www\.)?(\w+)(\.\w+)(/\w*)?")
        data = re.sub(regex, "", data)
        return data
    
    def rem_emails(self, data):
        regex = re.compile("([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")
        data = re.sub(regex, "", data)
        return data
    
    def rem_mention(self, data):
        regex = re.compile('@\w+')
        data = re.sub(regex, '', data)
        return data
    
    def rem_accent(self, data):
        data = unidecode(data)
        return data
    
    def rem_unicode(self, data):
        data = data.encode("ascii", "ignore").decode()
        return data
    
    def rem_punc(self, data):
        data = re.sub(f"[{string.punctuation}]", " ", data)
        return data
    
    def clean_numbers(self, data):
        data = re.sub('[0-9]{5,}', '#'*5, data)
        data = re.sub('[0-9]{4}', '#'*4, data)
        data = re.sub('[0-9]{3}', '#'*3, data)
        data = re.sub('[0-9]{2}', '#'*2, data)
        return data
    
    
    def rem_stopwords(self, data):
        return " ".join([word for word in str(data).split() if word not in self.stop_words])
    
    def rem_extra_space(self, data):
        data = re.sub(' +', ' ', data).strip()
        return data
    
    
    def lemmatize_data(self, data):
        words = [self.lemmatizer.lemmatize(word) for word in data.split()]
        data = ' '.join(words)
        return data
    
    def text_preprocessing(self, data):
        data['tweet'] = data['tweet'].str.lower()
        data['tweet'] = data['tweet'].apply(contractions.fix)
        data['tweet'] = data['tweet'].apply(self.rem_urls)
        data['tweet'] = data['tweet'].apply(self.rem_emails)
        data['tweet'] = data['tweet'].apply(lambda x: BeautifulSoup(x).get_text())
        data['tweet'] = data['tweet'].apply(self.rem_mention)
        data['tweet'] = data['tweet'].str.replace(':\(', 'sadness ')
        data['tweet'] = data['tweet'].str.replace(r':\)[$|\s]*', 'happiness ')
        data['tweet'] = data['tweet'].str.replace(r'\;\)[$|\s]*', 'happiness ')
        data['tweet'] = data['tweet'].apply(self.rem_accent)
        data['tweet'] = data['tweet'].apply(self.rem_unicode)
        data['tweet'] = data['tweet'].apply(self.rem_punc)
        data['tweet'] = data['tweet'].apply(self.clean_numbers)
        data['tweet'] = data['tweet'].apply(self.rem_stopwords)
        data['tweet'] = data['tweet'].apply(self.rem_extra_space)
        data['tweet'] = data['tweet'].apply(self.lemmatize_data)
        return data
        
    def __len__(self):
        return len(self.tweets)
    
    def __getitem__(self, idx):
        text = self.tweets.iloc[idx, -1]
        text = self.tokenizer.batch_encode_plus([text], return_tensors="pt", max_length=128, pad_to_max_length=True)['input_ids'].squeeze()
        label = [self.tweets.iloc[idx, -2]]
        label = torch.tensor(label)
        
        return text, label

In [495]:
def train_val_dataset(dataset, train_split=0.75, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), train_size=train_split, test_size=val_split)
    datasets = {'train':Subset(dataset, train_idx), 'val':Subset(dataset, val_idx)}
    return datasets

In [496]:
class textDatamodule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.tokenizer = None
    def setup(self, stage=None):
        data = textDataset("./nlp-getting-started/train.csv")
        self.tokenizer = data.tokenizer
        data = train_val_dataset(data)
        self.train_dataset = data['train']
        self.val_dataset = data['val']
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)    

In [497]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./check_point/',
    filename='{epoch}-{train_loss:.4f}-{val_loss:.4f}',
    monitor='val_loss',
    mode='min',
    save_top_k=2
)

In [498]:
wandb.init(
      mode='disabled',
      # Set the project where this run will be logged
      project="histopathologic-cancer-classifier", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"Test1", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0005,
      "data_size": 1,
      "batch_size":32,
      })
wandb_logger = WandbLogger()

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\pytorch_lightning\loggers\wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [499]:
trainer = pl.Trainer(accelerator='gpu',
                    devices=1,
                    max_epochs=100,
                    logger=wandb_logger,
                    callbacks=[checkpoint_callback]
                    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [500]:
dm = textDatamodule(batch_size=wandb.config['batch_size'])
dm.setup()

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_29048\1771628552.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(':\(', 'sadness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_29048\1771628552.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r':\)[$|\s]*', 'happiness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_29048\1771628552.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r'\;\)[$|\s]*', 'happiness ')


In [501]:
text_encoder = textClassifier(vocab_size=dm.tokenizer.vocab_size, embedding_dim=64, hidden_dim=32)

In [502]:
 trainer.fit(text_encoder, dm)

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_29048\1771628552.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(':\(', 'sadness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_29048\1771628552.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r':\)[$|\s]*', 'happiness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_29048\1771628552.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r'\;\)[$|\s]*', 'happiness ')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.